In [47]:
import git
import subprocess
import pandas as pd
from pandas import json_normalize
import json
from collections import defaultdict
import csv
import os

In [48]:
test_repository_path = "C:\\Users\\arman\\Academic 7th Sem\\Metrics\\TestRepositories\\refactoring-toy-example"
base_dir = "C:/Users/arman/Academic 7th Sem/Metrics/refactoring-productivity-analysis"

In [49]:
repo = git.Repo(test_repository_path)
commit_shas = [commit.hexsha for commit in repo.iter_commits()]
commit_shas[:10]

['d4bce13a443cf12da40a77c16c1e591f4f985b47',
 '9a5c33b16d07d62651ea80552e8782974c96bb8a',
 'a5a7f852e45c7cadc8d1524bd4d14a1e39785aa5',
 '0bb0526b70870d57cbac9fcc8c4a7346a4ce5879',
 'c286db365e7374fe4d08f54077abb7fba81dd296',
 'f3b628f8f9c71a2cdfa052025c4a1ed78ee4c45d',
 'ef3422578c0bcaef1561980ef077d06c3f6fc9f9',
 '1328d7873efe6caaffaf635424e19a4bb5e786a8',
 '3ce445fc6c67df447d30cc43556f204038e350c1',
 '0a46ed5c56c8b1576dfc92f3ec5bc2f0ea68aafe']

In [50]:
refactorig_miner_test_command = [f"{base_dir}/binaries/bin/RefactoringMiner.bat", "-h"]
refactorig_miner_test_command_mining = [f"{base_dir}/binaries/bin/RefactoringMiner.bat", 
                                        "-a",test_repository_path,"-json", f"{base_dir}/Dataset/testDataAllBranch.json"]

### Test Run of Refactoring miner

In [51]:
executor_process = subprocess.Popen(refactorig_miner_test_command_mining, stdout=subprocess.PIPE, shell=True, text=True)
output_text, _ = executor_process.communicate()
output_text

'Total count: [Commits: 38, Errors: 0, Refactorings: 84]\n'

In [52]:
# Specify the path to your JSON file
json_file_path = f"{base_dir}/Dataset/testDataAllBranch.json"
csv_file_path = f"{base_dir}/Dataset/refactoringDataset.csv"

In [ ]:
file_empty = not os.path.isfile(csv_file_path) or os.path.getsize(csv_file_path) == 0

with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

commit_refactoring_types = defaultdict(set)

In [ ]:
for commit in data['commits']:
    url = commit['url']

    refactoring_types = set()

    for refactoring in commit['refactorings']:
        refactoring_type = refactoring['type']

        refactoring_types.add(refactoring_type)

    commit_refactoring_types[url] = refactoring_types

In [ ]:
csv_data = []
csv_headers = ['url', 'Extract Method', 'Move Field', 'Inline Method', 'Move Method', 'Pull Up Method', 'Pull Up Field',
               'Extract Superclass', 'Extract Interface', 'Push Down Method', 'Push Down Field', 'total refactoring types']

for commit in data['commits']:
    url = commit['url']
    refactoring_types = commit_refactoring_types[url]

    row_data = [url]

    for refactoring_type in csv_headers[1:-1]:
        row_data.append(1 if refactoring_type in refactoring_types else 0)

    row_data.append(len(refactoring_types))
    csv_data.append(row_data)

In [67]:

# Check if the CSV file is empty
file_empty = not os.path.isfile(csv_file_path) or os.path.getsize(csv_file_path) == 0

# Load JSON data from file
with open(json_file_path, 'r') as json_file:
    data = json.load(json_file)

# Prepare data structures to store refactoring counts
commit_refactoring_types = defaultdict(set)

# Process each commit in the JSON data
for commit in data['commits']:
    url = commit['url']

    # Initialize a set to store unique refactoring types for each commit
    refactoring_types = set()

    # Process each refactoring in the commit
    for refactoring in commit['refactorings']:
        refactoring_type = refactoring['type']

        # Add refactoring type to the set
        refactoring_types.add(refactoring_type)

    # Update the set of refactoring types for the specific commit URL
    commit_refactoring_types[url] = refactoring_types

# Prepare data for writing to CSV
csv_data = []
csv_headers = ['url', 'Extract Method', 'Move Field', 'Inline Method', 'Move Method', 'Pull Up Method', 'Pull Up Field',
               'Extract Superclass', 'Extract Interface', 'Push Down Method', 'Push Down Field', 'total refactoring types']


# Append data for each commit
for commit in data['commits']:
    url = commit['url']
    refactoring_types = commit_refactoring_types[url]

    # Create a row for the CSV
    row_data = [url]

    # Append counts for each refactoring type
    for refactoring_type in csv_headers[1:-1]:
        row_data.append(1 if refactoring_type in refactoring_types else 0)

    # Append the total number of unique refactoring types
    row_data.append(len(refactoring_types))

    csv_data.append(row_data)

# Write data to CSV file
with open(csv_file_path, 'a', newline='') as csv_file:
    writer = csv.writer(csv_file)

    # Write header only if the file is empty
    if file_empty:
        writer.writerow(csv_headers)

    # Write data without duplicating URLs
    existing_urls = set()
    for row in csv_data:
        if row[0] not in existing_urls:
            writer.writerow(row)
            existing_urls.add(row[0])
